# Product Recommendations Using Object2Vec on Instacart Data

1. [Background](#Background)
1. [Data Exploration and Preparation](#Data-Exploration-and-Preparation)
1. [Create Product Recommendation Model](#Create-Product-Recommendation-Model)
1. [Product Retrieval in the Embedding Space](#Product-Retrieval-in-the-Embedding-Space)
1. [Clean Up](#Clean-Up)

# Background

### ObjectToVec
*Object2Vec* is a highly customizable multi-purpose algorithm that can learn embeddings of pairs of objects. Embeddings are an important feature engineering technique in machine learning (ML). They convert high dimensional vectors into low-dimensional space to make it easier to do machine learning with large sparse vector inputs. Embeddings also capture the semantics of the underlying data by placing similar items closer in the low-dimensional space. This makes the features more effective in training downstream models.

One of the well-known embedding techniques is Word2Vec, which provides embeddings for words. It has been widely used in many use cases, such as sentiment analysis, document classification, and natural language understanding. In addition to word embeddings, there are also use cases where we want to learn the embeddings of more general-purpose objects such as sentences, customers, and products. This is so we can build practical applications for information retrieval, product search, item matching, customer profiling based on similarity or as inputs for other supervised tasks. This is where Amazon SageMaker Object2Vec comes in, where the algorithm's embeddings are learned such that it preserves their pairwise **similarities** in the original space.
- **Similarity** is user-defined: users need to provide the algorithm with pairs of objects that they define as similar (1) or dissimilar (0); alternatively, the users can define similarity in a continuous sense (provide a real-valued similarity score)

- The learned embeddings can be used to efficiently compute nearest neighbors of objects, as well as to visualize natural clusters of related objects in the embedding space. In addition, the embeddings can also be used as features of the corresponding objects in downstream supervised tasks such as classification or regression

### In this notebook example:
We demonstrate how Object2Vec can be used to solve problems arising in recommendation systems. Specifically, the diagram below shows the customization of our model to the problem of predicting product recommendations, using a dataset that provides `(UserID, ProductID, Reordered)` samples.

#### Training with pairs of tokens: Collaborative recommendation system

Collaborative filtering is a popular technique for building recommendation systems. The main concept behind collaborative filtering is that users with similar tastes (based on observed user-item interactions) are more likely to have similar interactions with new items. Object2Vec can make recommendations by approximating the observed user-item interactions using low dimensional representations of users and items.

The following diagram shows how user-item interaction data can be used to learn the embedding of users and items. The resulting model can be used to predict user rating on a new item.

<img style="float:middle" src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2018/11/06/sagemaker-object2vec-4.gif" width="480">

### Dataset
- We use the Instacart Market Basket dataset: https://www.kaggle.com/c/instacart-market-basket-analysis/data

### Before Running the Notebook
- Please use a Python 3 kernel for the notebook
- Please make sure you have jsonlines package installed (if not, you can run the command below to install it):

In [ ]:
## Install jsonlines
!pip install --upgrade pip
!pip install jsonlines

In [ ]:
## Define important notebook variables
bucket = 'test-notebook-workshop'  #bucket name containing the uploaded Instacart Market Dataset
prefix = 'dataset'
 
# Import libraries
import boto3
import pandas as pd
import numpy as np
import re
from sagemaker import get_execution_role
import csv, jsonlines
import copy
import random
import struct
import io
import os

# Define IAM role
role = get_execution_role()

# Data Exploration and Preparation

Let's start by uploading the data sets from S3 and importing them as dataframes using pandas. We will then merge the data files and transform the data into a usable format for Object2Vec:

In [ ]:
## Main data sets we will utilize:
orders_dataFile = 'orders.csv'
orders_data_location = 's3://{}/{}'.format(bucket, prefix, orders_dataFile)
orders_data = pd.read_csv(orders_data_location)

products_dataFile = 'products.csv'
products_data_location = 's3://{}/{}'.format(bucket, prefix, products_dataFile)
products_data = pd.read_csv(products_data_location)

ordersP_dataFile = 'order_products__prior.csv'
ordersP_data_location = 's3://{}/{}'.format(bucket, prefix, ordersP_dataFile)
ordersP_data = pd.read_csv(ordersP_data_location)

In [ ]:
## Merge data files
orders_merge = pd.merge(ordersP_data,
               orders_data[['user_id', 'order_number', 'order_id']],
               on ='order_id')
products_merge = pd.merge(orders_merge,
               products_data[['product_id', 'product_name']],
               on ='product_id')
products_merge.head(10)

In [ ]:
## Drop unneeded columns
drop_data = products_merge.drop(['product_name', 'add_to_cart_order', 'order_number','order_id'], 1)
cols = drop_data.columns.tolist()
cols = cols[-1:] + cols[:-1]
final_data = drop_data[cols]

## For the purposes of this workshop, we will take a subset of the extremely large dataset to use for training.
subset_final_data = final_data.head(200000)
print(subset_final_data.head(10))

Let's also create some utility functions for further data exploration and preprocessing:

In [ ]:
## Some utility functions

def load_csv_data(filename, delimiter, verbose=True):
    """
    input: a file readable as csv and separated by a delimiter
    and has format users - products - reordered - etc
    output: a list, where each row of the list is of the form
    {'in0':userID, 'in1':productID, 'label':reordered}
    """
    to_data_list = list()
    users = list()
    products = list()
    reordered = list()
    unique_users = set()
    unique_products = set()
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter)
        for count, row in enumerate(reader):
            #if count!=0:
            to_data_list.append({'in0':[int(row[0])], 'in1':[int(row[1])], 'label':float(row[2])})
            users.append(row[0])
            products.append(row[1])
            reordered.append(float(row[2]))
    if verbose:
        print("In file {}, there are {} products".format(filename, len(products)))
    return to_data_list


def csv_to_augmented_data_dict(filename, delimiter):
    """
    Input: a file that must be readable as csv and separated by delimiter (to make columns)
    has format users - products - reordered - etc
    Output:
      Users dictionary: keys as user ID's; each key corresponds to a list of product bought by that user
      Products dictionary: keys as product ID's; each key corresponds a list of products bought by different users
    """
    to_users_dict = dict() 
    to_products_dict = dict()
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter)
        for count, row in enumerate(reader):
            #if count!=0:
            if row[0] not in to_users_dict:
                to_users_dict[row[0]] = [(row[1], row[2])]
            else:
                to_users_dict[row[0]].append((row[1], row[2]))
            if row[1] not in to_products_dict:
                to_products_dict[row[1]] = list(row[0])
            else:
                to_products_dict[row[1]].append(row[0])
    return to_users_dict, to_products_dict


def user_dict_to_data_list(user_dict):
    # turn user_dict format to data list format (acceptable to the algorithm)
    data_list = list()
    for user, product_reordered_list in user_dict.items():
        for product, reordered in product_reordered_list:
            data_list.append({'in0':[int(user)], 'in1':[int(product)], 'label':float(reordered)})
    return data_list

def divide_user_dicts(user_dict, sp_ratio_dict):
    """
    Input: A user dictionary, a ration dictionary
         - format of sp_ratio_dict = {'train':0.8, "test":0.2}
    Output: 
        A dictionary of dictionaries, with key corresponding to key provided by sp_ratio_dict
        and each key corresponds to a subdivded user dictionary
    """
    ratios = [val for _, val in sp_ratio_dict.items()]
    assert np.sum(ratios) == 1, "the sampling ratios must sum to 1!"
    divided_dict = {}
    for user, product_list in user_dict.items():
        sub_products_ptr = 0
        sub_products_list = []
        #mproduct_list, _ = zip(*product_rating_list)
        #print(product_list)
        for i, ratio in enumerate(ratios):
            if i < len(ratios)-1:
                sub_products_ptr_end = sub_products_ptr + int(len(product_list)*ratio)
                sub_products_list.append(product_list[sub_products_ptr:sub_products_ptr_end])
                sub_products_ptr = sub_products_ptr_end
            else:
                sub_products_list.append(product_list[sub_products_ptr:])
        for subset_name in sp_ratio_dict.keys():
            if subset_name not in divided_dict:
                divided_dict[subset_name] = {user: sub_products_list.pop(0)}
            else:
                #access sub-dictionary
                divided_dict[subset_name][user] = sub_products_list.pop(0)
    
    return divided_dict

def write_csv_to_jsonl(jsonl_fname, csv_fname, csv_delimiter):
    """
    Input: a file readable as csv and separated by delimiter (to make columns)
        - has format users - products - reordered - etc
    Output: a jsonline file converted from the csv file
    """
    with jsonlines.open(jsonl_fname, mode='w') as writer:
        with open(csv_fname, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=csv_delimiter)
            for count, row in enumerate(reader):
                #print(row)
                #if count!=0:
                writer.write({'in0':[int(row[0])], 'in1':[int(row[1])], 'label':float(row[2])})
        print('Created {} jsonline file'.format(jsonl_fname))
                    
    
def write_data_list_to_jsonl(data_list, to_fname):
    """
    Input: a data list, where each row of the list is a Python dictionary taking form
    {'in0':userID, 'in1':productID, 'label':reordered}
    Output: save the list as a jsonline file
    """
    with jsonlines.open(to_fname, mode='w') as writer:
        for row in data_list:
            #print(row)
            writer.write({'in0':row['in0'], 'in1':row['in1'], 'label':row['label']})
    print("Created {} jsonline file".format(to_fname))

def data_list_to_inference_format(data_list, binarize=True, label_thres=0.5):
    """
    Input: a data list
    Output: test data and label, acceptable by SageMaker for inference
    """
    data_ = [({"in0":row['in0'], 'in1':row['in1']}, row['label']) for row in data_list]
    data, label = zip(*data_)
    infer_data = {"instances":data}
    if binarize:
        label = get_binarized_label(list(label), label_thres)
    return infer_data, label


def get_binarized_label(data_list, thres):
    """
    Input: data list
    Output: a binarized data list for recommendation task
    """
    for i, row in enumerate(data_list):
        if type(row) is dict:
            #if i < 10:
                #print(row['label'])
            if row['label'] > thres:
                #print(row)
                data_list[i]['label'] = 1
            else:
                data_list[i]['label'] = 0
        else:
            if row > thres:
                data_list[i] = 1
            else:
                data_list[i] = 0
    return data_list

In [ ]:
## Load data and split into test/train/val

y = subset_final_data.product_id 
X = subset_final_data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
## Export Data to S3

training_data_bucket = 'test-notebook-workshop'  #your bucket name that you created earlier in the console

from io import StringIO
DESTINATION = training_data_bucket

def _write_dataframe_to_csv_on_s3(dataframe, filename):
    #""" Write a dataframe to a CSV on S3 """
    print("Writing {} records to {}".format(len(dataframe), filename))
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep="\t", index=False, header=False)
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(DESTINATION, filename).put(Body=csv_buffer.getvalue())

_write_dataframe_to_csv_on_s3(X_train, 'final_training_data')
_write_dataframe_to_csv_on_s3(X_val, 'final_validation_data')
_write_dataframe_to_csv_on_s3(products_data, 'products_data')

In [ ]:
## Sync files from S3 to local SageMaker instance

#enter your bucket name from above = training_data_bucket
!aws s3 sync s3://retail-analytics-workshop-<your-initials> {training-data} #your bucket name from above

In [ ]:
## Load data and shuffle:

prefix = '/home/ec2-user/SageMaker/{training-data}'
train_path = os.path.join(prefix, 'final_training_data')
val_path = os.path.join(prefix, 'final_validation_data')

train_data_list = load_csv_data(train_path, '\t')
random.shuffle(train_data_list)
validation_data_list = load_csv_data(val_path, '\t')
random.shuffle(validation_data_list)

In [ ]:
to_users_dict, to_products_dict = csv_to_augmented_data_dict(train_path, '\t')

In [ ]:
## Save training and validation data locally for recommendation (classification) task

### Binarize the data

train_c = get_binarized_label(copy.deepcopy(train_data_list), 3.0)
valid_c = get_binarized_label(copy.deepcopy(validation_data_list), 3.0)

write_data_list_to_jsonl(train_c, 'train_c.jsonl')
write_data_list_to_jsonl(valid_c, 'validation_c.jsonl')

# Create Product Recommendation Model

In this section, we showcase how to use Object2Vec to recommend products. Here, if a product for a given user is binarized to 1, then it means that the product should be recommended to the user; otherwise, the label is binarized to 0. The binarized data set is already obtained in the preprocessing section, so we will proceed to apply the algorithm.

### Upload Data to S3
We upload the binarized datasets for classification task to S3:

In [ ]:
input_prefix = 'object2vec/retail/input'
output_prefix = 'object2vec/retail/output'

from sagemaker.session import s3_input

s3_client = boto3.client('s3')
input_paths = {}
output_path = os.path.join('s3://', training_data_bucket, output_prefix)

for data_name in ['train', 'validation']:
    fname = '{}_c.jsonl'.format(data_name)
    pre_key = os.path.join(input_prefix, 'recommendation', f"{data_name}")
    data_path = os.path.join('s3://', training_data_bucket, pre_key, fname)
    s3_client.upload_file(fname, training_data_bucket, os.path.join(pre_key, fname))
    input_paths[data_name] = s3_input(data_path, distribution='ShardedByS3Key', content_type='application/jsonlines')
    print('Uploaded data to {}'.format(data_path))

print('Trained model will be saved at', output_path)


### Get ObjectToVec Algorithm Image

In [ ]:
## Get obj2vec image
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()

role = get_execution_role()
print(role)

## Get docker image of ObjectToVec algorithm
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'object2vec')

### Train the Model

We first define training hyperparameters. To learn more about SageMaker's Object2Vec hyperparameters, please visit our documentation page:
- https://docs.aws.amazon.com/sagemaker/latest/dg/object2vec-hyperparameters.html

In [ ]:
from sagemaker.session import s3_input

## Define Object2Vec hyperparameters:

hyperparameters_c = {
    "_kvstore": "device",
    "_num_gpus": "auto",
    "_num_kv_servers": "auto",
    "bucket_width": 0,
    "early_stopping_patience": 1, 
    "early_stopping_tolerance": 0.01,
    "enc0_cnn_filter_width": 3,
    "enc0_layers": "auto",
    "enc0_max_seq_len": 1,
    "enc0_network": "pooled_embedding",
    "enc0_token_embedding_dim": 300,
    "enc0_vocab_size": 220000,
    "enc1_cnn_filter_width": 3,
    "enc1_layers": "auto",
    "enc1_max_seq_len": 1,
    "enc1_network": "pooled_embedding",
    "enc1_token_embedding_dim": 300,
    "enc1_vocab_size": 220000,
    "enc_dim": 2048,
    "epochs": 1,
    "learning_rate": 0.001,
    "mini_batch_size": 200,
    "mlp_activation": "relu",
    "mlp_dim": 1024,
    "mlp_layers": 1,
    "num_classes": 2,
    "optimizer": "adam",
    "output_layer": "softmax"
}

In [ ]:
## Get estimator
classifier = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.24xlarge',
                                    output_path=output_path,
                                    sagemaker_session=sess)

## Set hyperparameters
classifier.set_hyperparameters(**hyperparameters_c)

## Train, tune, and test the model (training runtime: ~25 min)
classifier.fit(input_paths)

Next, we can create, deploy, and validate the model after training:

In [ ]:
from sagemaker.predictor import json_serializer, json_deserializer

## Create a model using the trained algorithm
classification_model = classifier.create_model(
                        serializer=json_serializer,
                        deserializer=json_deserializer,
                        content_type='application/json')

## Deploy the model (~12 min.)
predictor = classification_model.deploy(initial_instance_count=1, instance_type='ml.m5.4xlarge')

In [ ]:
valid_c_data, valid_c_label = data_list_to_inference_format(copy.deepcopy(validation_data_list), 
                                                            label_thres=0.5, binarize=True)
predictions = predictor.predict(valid_c_data)

In [ ]:
def get_class_accuracy(res, labels, thres):
    if type(res) is dict:
        res = res['predictions']
    assert len(res)==len(labels), 'result and label length mismatch!'
    accuracy = 0
    for row, label in zip(res, labels):
        if type(row) is dict:
            if row['scores'][1] > thres:
                prediction = 1
            else: 
                prediction = 0
            if label > thres:
                label = 1
            else:
                label = 0
            accuracy += 1 - (prediction - label)**2
    return accuracy / float(len(res))

print("The accuracy on the binarized validation set is %.3f" %get_class_accuracy(predictions, valid_c_label, 0.5))

# Product Retrieval in the Embedding Space
Since Object2Vec transforms user and product ID's into embeddings as part of the training process - after training, it obtains user and product embeddings in the left and right encoders, respectively. Intuitively, the embeddings should be tuned by the algorithm in a way that facilitates the supervised learning task: since for a specific user, popular products should have been reordered, we expect that similar products that users would be interested in buying should be **close-by** in the embedding space.

In this section, we demonstrate how to find the nearest-neighbor (in Euclidean distance) of a given product ID, among all product ID's in our subset of data.

Let's first create some utility functions for this task:

In [ ]:
def get_product_embedding_dict(product_ids, trained_model):
    input_instances = list()
    for s_id in product_ids:
        input_instances.append({'in1': [s_id]})
    data = {'instances': input_instances}
    product_embeddings = trained_model.predict(data)
    embedding_dict = {}
    for s_id, row in zip(product_ids, product_embeddings['predictions']):
        embedding_dict[s_id] = np.array(row['embeddings'])
    return embedding_dict


def get_nn_of_product(product_id, candidate_product_ids, embedding_dict):
    product_emb = embedding_dict[product_id]
    min_dist = float('Inf')
    best_id = candidate_product_ids[0]
    for idx, m_id in enumerate(candidate_product_ids):
        candidate_emb = embedding_dict[m_id]
        curr_dist = np.linalg.norm(candidate_emb - product_emb)
        if curr_dist < min_dist:
            best_id = m_id
            min_dist = curr_dist
    return best_id, min_dist


def get_unique_product_ids(data_list):
    unique_product_ids = set()
    for row in data_list:
        unique_product_ids.add(row['in1'][0])
    return list(unique_product_ids)

In [ ]:
train_data_list = load_csv_data(train_path, '\t', verbose=False)
unique_product_ids = get_unique_product_ids(train_data_list)
embedding_dict = get_product_embedding_dict(unique_product_ids, predictor)
candidate_product_ids = unique_product_ids.copy()

Using the script below, you can check out what is the closest product to any product in the data set. Last time we ran it, the closest product to Carrots in the embedding space was Organic Kale. Note that, the result will likely differ slightly across different runs of the algorithm, due to randomness in initialization of model parameters.

However, let's plug in the product id for Carrots, 17794, that we want to examine and validate this prior recommendation (you can find the product name and ID pair in the products_data file).

In [ ]:
#product_id_to_examine = '<product id for carrots>'
product_id_to_examine = 17794
products_data.loc[products_data['product_id'] == 17794]

Now, let's find the nearest neighbor to Carrots in the embedding space and find the product's name from our dataset using it's product ID:

In [ ]:
prefix = '/home/ec2-user/SageMaker/{training-data}'
product_data_path = os.path.join(prefix, 'final_training_data')
candidate_product_ids.remove(product_id_to_examine)
best_id, min_dist = get_nn_of_product(product_id_to_examine, candidate_product_ids, embedding_dict)
products_data.loc[products_data['product_id'] == best_id]

Therefore, we can see that our model has correctly recommended to buy Michigan Organic Kale, for those users who have bought Carrots in the past. Both being vegetables, we can understand why we would recommend Kale to customers who previously bought Carrots.

# Clean Up

It is recommended to always delete the endpoints used for hosting the model:

In [ ]:
## Delete model endpoint
sess.delete_endpoint(predictor.endpoint)